In [1]:
from moviepy.editor import VideoFileClip
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import scipy
import time
from tqdm import tqdm

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

plt.style.use('dark_background')

In [3]:
hog_descriptor = cv2.HOGDescriptor((64,64), (16,16), (8,8), (8,8), 9)

def compute_color_hist(img, nbins=32, bins_range=(0, 256)):
    features = []
    for channel in range(img.shape[2]):
        hist = cv2.calcHist([img], [channel], None, [nbins], bins_range)
        features.append(hist[0])
    return np.ravel(features)
    
def compute_bin_spatial(img, size=(32, 32)):
    return cv2.resize(img, size)

def compute_hog(img, shape=(-1,)):
    hogs = []
    for channel in range(img.shape[2]):
        hogs.append(hog_descriptor.compute(img[:,:,channel]).reshape(shape))
    return hogs

def extract_features(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    hog0, hog1, hog2 = compute_hog(img)
    features = [compute_color_hist(img), compute_bin_spatial(img).ravel(), hog0, hog1, hog2]
    return np.concatenate(features)

In [4]:
vehicle = []
for img_path in tqdm(glob.glob('vehicles/**/*.png')):
    img = cv2.imread(img_path)
    vehicle.append(extract_features(img))

nonvehicle = []
for img_path in tqdm(glob.glob('non-vehicles/**/*.png')):
    img = cv2.imread(img_path)
    nonvehicle.append(extract_features(img))

100%|██████████| 8971/8971 [00:04<00:00, 1814.78it/s]

100%|█████████▉| 8951/8971 [00:04<00:00, 1809.46it/s]

 98%|█████████▊| 8769/8971 [00:04<00:00, 1805.98it/s]

 96%|█████████▌| 8587/8971 [00:04<00:00, 1800.86it/s]

 94%|█████████▎| 8406/8971 [00:04<00:00, 1801.91it/s]

...

  6%|▌         | 532/8792 [00:00<13:46, 10.00it/s]

  4%|▍         | 353/8792 [00:00<20:03,  7.01it/s]

  2%|▏         | 177/8792 [00:00<29:13,  4.91it/s]

  0%|          | 1/8792 [00:00<42:34,  3.44it/s]

  0%|          | 0/8792 [00:00<?, ?it/s]

In [5]:
X = np.concatenate([vehicle, nonvehicle]).astype(np.float32)
X_scaler = StandardScaler().fit(X)
X = X_scaler.transform(X)

y = np.array([1] * len(vehicle) + [0] * len(nonvehicle))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
class Svm:
    def __init__(self):
        self.svm = cv2.ml.SVM_create()
        self.svm.setKernel(cv2.ml.SVM_LINEAR)
        self.svm.setType(cv2.ml.SVM_C_SVC)
        self.svm.setC(0.0001)

    def train(self, X_train, y_train):
        self.svm.train(X_train, cv2.ml.ROW_SAMPLE, y_train)

    def score(self, X_test, y_test):
        return accuracy_score(y_test, self.predict(X_test))

    def predict(self, data):
        if len(data.shape)==1:
           data = np.array([data])
        return self.svm.predict(data)[1].astype(np.int).ravel()

In [7]:
clf = Svm()
clf.train(X_train, y_train)
print('Test Accuracy of SVM = ', round(clf.score(X_test, y_test), 4))

Test Accuracy of SVM =  0.9921


In [8]:
def slide_window(img, x_start=0, y_start=0, scale=1, step=1, window_size=64, cell_size=8, feature_size=1764):
    hog_width = img.shape[1] // cell_size - window_size // cell_size + 1
    hog_height = img.shape[0] // cell_size - window_size // cell_size + 1

    hog0, hog1, hog2 = compute_hog(img, (hog_height, hog_width, feature_size))

    spatial = compute_bin_spatial(img, (int(img.shape[1] / 2), int(img.shape[0] / 2)))

    windows = []
    for ys in range(0, hog_height, step):
        for xs in range(0, hog_width, step):
            x_ch = xs * cell_size
            y_ch = ys * cell_size
            ch = compute_color_hist(img[y_ch:y_ch+window_size,x_ch:x_ch+window_size])

            x_spatial = xs * cell_size // 2
            y_spatial = ys * cell_size // 2
            sub_spatial = spatial[y_spatial:y_spatial+window_size//2,x_spatial:x_spatial+window_size//2].ravel()
            
            features = np.concatenate([ch, sub_spatial, hog0[ys, xs], hog1[ys, xs], hog2[ys, xs]])
            features = X_scaler.transform(features.reshape(1, -1))
            if clf.predict(features):
                x = int(x_start + xs * cell_size * scale)
                y = int(y_start + ys * cell_size * scale)
                windows.append(((x, y), (x + int(window_size * scale), y + int(window_size * scale))))

    return windows

In [9]:
def search_scaled(img, x_start=0, y_start=400, scales=((1,524,1),(1.5,588,2),(2,652,3))):
    orig_img = img

    windows = []
    for scale, y_stop, step in scales:
        img = orig_img[y_start:y_stop,x_start:]
        img = cv2.resize(img, (int(img.shape[1] / scale), int(img.shape[0] / scale)))
        windows.extend(slide_window(img, x_start, y_start, scale, step))

    return windows

In [10]:
def search_heated(img, threshold=3):
    from scipy.ndimage.measurements import label
    
    heatmap = np.zeros_like(img)
    
    for box in search_scaled(img):
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    heatmap[heatmap <= threshold] = 0
    labels = label(heatmap)

    boxes = []
    for car_number in range(1, labels[1]+1):
        nonzero = (labels[0] == car_number).nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        boxes.append(((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy))))

    return boxes

In [11]:
def find_cars(img, source='RGB', color=(0,0,255)):
    orig_img = img
    img = cv2.cvtColor(img, eval('cv2.COLOR_' + source + '2YCrCb'))

    for bbox in search_heated(img):
        cv2.rectangle(orig_img, bbox[0], bbox[1], color, 6)
        
    return orig_img

In [12]:
clip = VideoFileClip('project_video.mp4')
clip = clip.fl_image(find_cars)
%time clip.write_videofile('output.mp4', audio=False)

CPU times: user 5min 15s, sys: 2.1 s, total: 5min 17s
Wall time: 5min 10s


[MoviePy] >>>> Video ready: output.mp4 



[MoviePy] Done.


100%|█████████▉| 1260/1261 [05:10<00:00,  3.95it/s]

100%|█████████▉| 1259/1261 [05:09<00:00,  3.98it/s]

...

  0%|          | 2/1261 [00:00<05:13,  4.02it/s]

  0%|          | 1/1261 [00:00<05:16,  3.98it/s]

  0%|          | 0/1261 [00:00<?, ?it/s]

[MoviePy] Writing video output.mp4


[MoviePy] >>>> Building video output.mp4
